In [1]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt, floor, ceil
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datetime import timedelta

In [2]:
%load_ext line_profiler

In [3]:
data_path = 'data/'

In [4]:
data = pd.read_pickle(f'{data_path}processed/timeseries_2018_through_2023.pkl').reset_index(drop=True)

In [5]:
data['ZCTA_ID'] = data.group.str.slice(0, -4)

In [6]:
data['year'] = data['date_dateobj'].dt.year
data['month'] = data['date_dateobj'].dt.month
data['day'] = data['date_dateobj'].dt.day

In [7]:
pd.set_option('display.max_rows', 300)

In [8]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of Earth in kilometers. Use 3956 for miles
    return c * r


In [9]:
def create_distance_matrix(df, identifier='ZCTA_ID', distance_metric=haversine):
    """
    Compute distance from ZCTA1 to ZCTA2 for all ZCTAs using the haversine formula
    """
    df[identifier] = df['group'].str.slice(0, -4)
    unique_zctas = df[[identifier, 'zcta_centroid_longitude', 'zcta_centroid_latitude']].drop_duplicates().set_index(identifier)
    
    num_zctas = len(unique_zctas)
    distance_matrix = pd.DataFrame(index=unique_zctas.index, columns=unique_zctas.index, dtype=float)
    for zcta1 in unique_zctas.index:
        for zcta2 in unique_zctas.index:
            if zcta1 == zcta2:
                distance_matrix.loc[zcta1, zcta2] = 0.0  # Include self as a nearest ZCTA
            else:
                dist = distance_metric(unique_zctas.loc[zcta1, 'zcta_centroid_longitude'], unique_zctas.loc[zcta1, 'zcta_centroid_latitude'],
                                 unique_zctas.loc[zcta2, 'zcta_centroid_longitude'], unique_zctas.loc[zcta2, 'zcta_centroid_latitude'])
                distance_matrix.loc[zcta1, zcta2] = dist

    return distance_matrix

In [10]:
# Precompute distance matrix from ZCTA1 to ZCTA2
distance_matrix = create_distance_matrix(data)

In [11]:
distance_matrix

ZCTA_ID,01060,01201,01247,01364,01451,01462,01503,01776,02038,02155,...,97731,97801,97874,98053,98056,98118,98538,98577,98665,98847
ZCTA_ID,,,,,,,,,,,,,,,,,,,,,
01060,0.000000,54.855507,55.448935,42.625406,89.068199,80.463490,82.111198,99.496849,103.861124,125.565549,...,3957.739350,3661.676706,3762.821603,3861.847657,3877.369329,3882.641172,3971.328569,4001.482164,3953.481768,3764.470775
01201,54.855507,0.000000,31.690612,82.642589,139.755625,128.508760,134.903663,152.285277,158.711345,177.902358,...,3902.973714,3607.204113,3708.214988,3808.007691,3823.487555,3828.771598,3917.188318,3947.403095,3899.132456,3710.511426
01247,55.448935,31.690612,0.000000,65.055594,125.160829,111.865014,123.312641,140.047022,152.867780,164.250675,...,3909.866935,3612.198508,3713.926445,3810.639123,3826.246283,3831.492653,3920.801881,3950.800887,3903.559047,3713.517452
01364,42.625406,82.642589,65.055594,0.000000,60.156835,46.931173,59.578657,75.627458,93.130687,99.267928,...,3973.953844,3675.499418,3777.550027,3872.638940,3888.323861,3893.546940,3983.377460,4013.253031,3966.571240,3775.748736
01451,89.068199,139.755625,125.160829,60.156835,0.000000,15.491943,14.008461,17.891005,48.044896,39.111989,...,4033.724594,3734.765886,3837.030366,3930.979281,3946.720943,3951.926861,4042.133796,4071.920267,4025.636195,3834.260003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98118,3882.641172,3828.771598,3831.492653,3893.546940,3951.926861,3936.859971,3953.122077,3968.708828,3984.358438,3989.987602,...,490.247465,336.125658,358.744117,23.175777,6.796294,0.000000,136.057077,142.455449,209.261488,122.400974
98538,3971.328569,3917.188318,3920.801881,3983.377460,4042.133796,4027.170772,4042.947881,4058.740854,4073.587253,4080.424603,...,385.556461,346.400240,317.755149,157.534365,136.335255,136.057077,0.000000,39.306928,96.530697,218.246611
98577,4001.482164,3947.403095,3950.800887,4013.253031,4071.920267,4056.930356,4072.828802,4088.571880,4103.616716,4110.156378,...,414.053587,384.591876,356.770547,165.445554,144.580919,142.455449,39.306928,0.000000,129.561984,241.278398


In [12]:
def find_nearest_zctas(base_df, dist_matrix, nearest_zctas_lookup, target_zcta_id, target_date, n=1, n_years_back=3):
    """
    Precomputes nearest ZCTAs within max_distance_km to a given ZCTA as a dict of str -> list
    """    
    # Convert valid ZCTAs list to DataFrame for joining
    valid_zctas_df = pd.DataFrame({'ZCTA_ID': nearest_zctas_lookup[target_zcta_id]})
    
    # Calculate the range of years to include (previous three years)
    years_to_include = range(target_date.year - n_years_back, target_date.year)

    # Filter entries that match the day and month and are within the last three years
    base_df = base_df[
        (base_df['month'].values == target_date.month) &
        (base_df['day'].values == target_date.day) &
        (base_df['year'].isin(years_to_include))
    ]

    # Perform a merge (join) on ZCTA_ID
    nearest_zctas = base_df.merge(valid_zctas_df, on='ZCTA_ID')
    nearest_zctas = nearest_zctas.assign(distance=nearest_zctas['ZCTA_ID'].map(dist_matrix.loc[target_zcta_id]))

    # Sort by distance and then by year (descending to prioritize recent years)
    nearest_zctas = nearest_zctas.sort_values(by=['distance', 'date_dateobj'], ascending=[True, False]).head(n)

    return nearest_zctas

In [14]:
def precompute_nearest_zctas(dist_matrix, max_distance_km=3000):
    nearest_zctas = {}
    for zcta_id in dist_matrix.index:
        # Filter distances that are less than or equal to max_distance_km
        within_distance = dist_matrix.loc[zcta_id] <= max_distance_km
        valid_neighbors = dist_matrix.columns[within_distance]
        nearest_zctas[zcta_id] = valid_neighbors.tolist()
    return nearest_zctas

In [15]:
# Precompute nearest ZCTAs as a lookup table
max_distance = 300
nearest_zctas_lookup = precompute_nearest_zctas(distance_matrix, max_distance)

In [17]:
def custom_round(number):
    if number > 0:
        return floor(number + 0.5)
    else:
        return ceil(number - 0.5)

In [18]:
def model_prediction(nearest_zctas):
    if nearest_zctas.empty:
        return 0  # Default prediction if no nearby ZCTAs
    return custom_round(nearest_zctas['final_target'].mean())

In [30]:
test_zcta = '01033'
test_year = 2023

In [31]:
find_nearest_zctas(data, distance_matrix, nearest_zctas_lookup, test_zcta, pd.Timestamp(f'{test_year}-05-07'), n=5, n_years_back=3)

,group,date_dateobj,time_idx,ppt,tmax,tmin,tavg,day_length_seconds,final_target,zcta_centroid_longitude,zcta_centroid_latitude,ZCTA_ID,year,month,day,distance
169,010332022,2022-05-07,127,0.000,17.318,8.675,12.996,51469.86,0,-72.503625,42.260809,01033,2022,5,7,0.000000
65,010752021,2021-05-07,127,0.000,17.783,4.264,11.023,51501.67,0,-72.579228,42.256743,01075,2021,5,7,6.238316
61,010072021,2021-05-07,127,0.000,17.327,4.283,10.805,51507.98,1,-72.400360,42.278750,01007,2021,5,7,8.727975
0,010072020,2020-05-07,128,0.103,14.548,1.968,8.258,51540.90,1,-72.400360,42.278750,01007,2020,5,7,8.727975
63,010352021,2021-05-07,127,0.000,17.539,4.351,10.945,51530.36,0,-72.569208,42.355636,01035,2021,5,7,11.843389


In [32]:
data[(data['ZCTA_ID'] == test_zcta) & (data['year'] == 2022)]

,group,date_dateobj,time_idx,ppt,tmax,tmin,tavg,day_length_seconds,final_target,zcta_centroid_longitude,zcta_centroid_latitude,ZCTA_ID,year,month,day
652750,010332022,2022-01-01,1,0.564,7.808,4.411,6.109,32941.78,0,-72.503625,42.260809,01033,2022,1,1
652751,010332022,2022-01-02,2,6.159,9.826,6.666,8.246,32990.81,0,-72.503625,42.260809,01033,2022,1,2
652752,010332022,2022-01-03,3,0.416,7.824,-4.407,1.709,33043.87,0,-72.503625,42.260809,01033,2022,1,3
652753,010332022,2022-01-04,4,0.000,-4.138,-11.567,-7.852,33100.92,0,-72.503625,42.260809,01033,2022,1,4
652754,010332022,2022-01-05,5,0.000,0.519,-11.728,-5.604,33161.89,0,-72.503625,42.260809,01033,2022,1,5
652755,010332022,2022-01-06,6,2.213,4.402,-3.131,0.635,33226.74,0,-72.503625,42.260809,01033,2022,1,6
652756,010332022,2022-01-07,7,6.642,1.697,-4.010,-1.156,33295.39,0,-72.503625,42.260809,01033,2022,1,7
652757,010332022,2022-01-08,8,1.769,-0.601,-12.428,-6.514,33367.79,0,-72.503625,42.260809,01033,2022,1,8
652758,010332022,2022-01-09,9,0.000,-1.439,-12.360,-6.900,33443.87,0,-72.503625,42.260809,01033,2022,1,9
652759,010332022,2022-01-10,10,1.739,2.327,-4.409,-1.041,33523.55,0,-72.503625,42.260809,01033,2022,1,10


In [33]:
def predict_for_zcta(base_df, dist_matrix, nearest_zctas_lookup, target_zcta_id, target_date, n=3, n_years_back=3):
    nearest_zctas = find_nearest_zctas(base_df, dist_matrix, nearest_zctas_lookup, target_zcta_id, target_date, n, n_years_back)
    prediction = model_prediction(nearest_zctas)
    return prediction

## Assess various baseline models to see which performs best

### Baseline model computation logic

The baseline models are computed as follows.

We have 5 years of data. We will compute a baseline target for `2021-01-01`-`2023-06-30`. 

The logic to compute these baseline models contains two variables:
    1. n - the number of nearby ZCTAs we want to include in our computation
    2. max_distance_km - the maximum distance we are willing to consider looking for other counties
    
We grid searched over these parameters to find a baseline model which performs best with regard to
F1 score/recall/precision.
    
The procedure to compute a baseline model is as follows:
  1. For each observation (consisting of a ZCTA/day), collect n 'nearby ZCTA observations' from PREVIOUS YEARS ONLY. Procedure to get 'nearby ZCTA observations':
    1. First, rank previous observations by distance (including from same ZCTA). 
    2. Then, rank previous observations by year (most recent years get priority). 
    3. Take n of these previous observations
  2. For each of the nearby observations computed, take a majority vote on the label on a given date. The majority vote indicates the status of the predicted label. 

In [34]:
for idx in range(1,30):
    date_of_interest = pd.Timestamp(f'2021-05-{str(idx).zfill(2)}')
    prediction = predict_for_zcta(data, distance_matrix, nearest_zctas_lookup, '01060', date_of_interest, n=5, n_years_back=3)
    print(f"Predicted final_target: {prediction}")

Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 1
Predicted final_target: 1
Predicted final_target: 1
Predicted final_target: 1
Predicted final_target: 1
Predicted final_target: 0
Predicted final_target: 1
Predicted final_target: 1
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0
Predicted final_target: 0


In [35]:
def evaluate_predictions(y_true, y_pred):
    """
    Evaluate model predictions using accuracy, precision, and recall.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return accuracy, precision, recall, f1

In [44]:
def perform_grid_search(base_df, dist_matrix, params):
    """
    Perform grid search over specified parameters for n and max_distance_km.
    """
    results = []
    for n in params['n']:
        for max_distance in params['max_distance_km']:
            
            # Precompute nearest ZCTAs with different max_distance
            nearest_zctas_lookup = precompute_nearest_zctas(distance_matrix, max_distance)
            
            predictions = []
            true_values = []
            # Assume all entries are to be used for evaluation
            for index, row in base_df.iterrows():
                pred = predict_for_zcta(base_df, dist_matrix, nearest_zctas_lookup, row['ZCTA_ID'], row['date_dateobj'], n=n, n_years_back=3)
                predictions.append(pred)
                true_values.append(row['final_target'])
            
            # Evaluate predictions
            accuracy, precision, recall, f1 = evaluate_predictions(true_values, predictions)
            results.append({
                'n': n,
                'max_distance_km': max_distance,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1
            })
            base_df[f'pred_n_{str(n)}_max_distance_{str(max_distance)}'] = pd.Series(predictions)
    
    return pd.DataFrame(results), base_df


In [45]:
# Define parameter grid
# params = {
#     'n': [1, 2, 3, 4],
#     'max_distance_km': [10, 50, 100, 200]
# }
params = {
    'n': [1,2,3,4,5],
    'max_distance_km': [300]
}


# Assuming the functions and data are all set
results_df, df_with_preds = perform_grid_search(data, distance_matrix, params)

In [53]:
# Convert to DataFrame and display in bold
def bold_max(s):
    is_max = s == s.max()
    return ['font-weight: bold;'.format(val) if max_val else ''
            for val, max_val in zip(s, is_max)]

In [51]:
df_with_preds['guess_not_in_season'] = 0

In [61]:
relevant_predictions = df_with_preds[df_with_preds['date_dateobj'] > '2020-07-01'].reset_index(drop=True)

In [62]:
results = []
for n in params['n']:
    for max_distance in params['max_distance_km']:
        col_name = f'pred_n_{str(n)}_max_distance_{str(max_distance)}'
        accuracy, precision, recall, f1 = evaluate_predictions(relevant_predictions['final_target'], relevant_predictions[col_name])
        results.append({
            'name': col_name,
            'n': n,
            'max_distance_km': max_distance,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
        
accuracy, precision, recall, f1 = evaluate_predictions(relevant_predictions['final_target'], relevant_predictions['guess_not_in_season'])
results.append({
    'name': 'Predicting "no"',
    'n': None,
    'max_distance_km': None,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1
})

true_results_df = pd.DataFrame(results)
result_styled = true_results_df.style.apply(bold_max, subset=['accuracy', 'precision', 'recall', 'f1'])
result_styled

,name,n,max_distance_km,accuracy,precision,recall,f1
0,pred_n_1_max_distance_300,1.000000,300.000000,0.917713,0.266786,0.270894,0.268824
1,pred_n_2_max_distance_300,2.000000,300.000000,0.896619,0.256899,0.449844,0.327034
2,pred_n_3_max_distance_300,3.000000,300.000000,0.929935,0.308122,0.204534,0.245862
3,pred_n_4_max_distance_300,4.000000,300.000000,0.920768,0.296857,0.306080,0.301398
4,pred_n_5_max_distance_300,5.000000,300.000000,0.934318,0.320578,0.157447,0.211178
5,"Predicting ""no""",nan,nan,0.944160,0.000000,0.000000,0.000000


In [63]:
relevant_predictions.to_pickle(f'{data_path}processed/timeseries_2021_through_2023_with_baseline1.pkl')

In [64]:
df_with_preds.to_pickle(f'{data_path}processed/timeseries_2018_through_2023_with_baseline1.pkl')

In [ ]:
%lprun -f find_nearest_zctas [find_nearest_zctas(data, distance_matrix, nearest_zctas_lookup, test_zcta, pd.Timestamp(f'{test_year}-05-07'), n=5, n_years_back=3) for idx in range(100)]

In [ ]:
# (Numpy filter) Total time: 8.01443 s
# OG method: (pandas filter) Total time: 0.576382 s
# With query method: Total time: 0.520413 s